In [3]:
with open("przyklad.py") as f:
    print(f.read())
    # wewnatrz
# na zewnatrz

"""Krótki przykład introspekcji przestrzeni nazw w Pythonie."""

a = 1

if __name__ == "__main__":
    print("Zakres dostępnych nazw:", dir())
    print("Zmienne lokalne modułu:", locals())
    print("Zmienne globalne modułu:", {k: type(v).__name__ for k, v in globals().items()})



## Context manager

In [8]:
class ManagedFile:
    def __init__(self, filename):
        self.filename = filename

    def __enter__(self):
        print(f"Otwieram plik {self.filename}...")
        self.file = open(self.filename)
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        print("Zamykam plik")
        self.file.close()
        return False

with ManagedFile("przyklad.py") as f:
    1/0
    print(f.read())

Otwieram plik przyklad.py...
Zamykam plik


ZeroDivisionError: division by zero

In [7]:
try:
    f = open("przyklad.py")
    1/0
finally:
    print("zamykam")
    f.close()
    

zamykam


ZeroDivisionError: division by zero

## iterator

In [9]:
# start -> 0

In [10]:
class Countdown:

    def __init__(self, start: int):
        self.current = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < 1:
            raise StopIteration

        val = self.current
        self.current -= 1
        return val

for i in Countdown(5):
    print(i)
        

5
4
3
2
1


In [11]:
c = Countdown(5)

next(c)

5

In [16]:
next(c)

StopIteration: 

## sekwencje / container

In [40]:
class PositiveIntegerList:
    def __init__(self, data):
        self._data = list(data)

    def __getitem__(self, index):
        return self._data[index]

    def __len__(self):
        return len(self._data)

    def __contains__(self, item):
        return item in self._data

    def append(self, item):
        if not isinstance(item, int):
            raise ValueError("wartosci dozwolone to tylko inty")

        if item < 0:
            raise ValueError("wartosci musza byc dodanie")

        self._data.append(item)

In [18]:
lista = [1, 2, 3]


In [19]:
lista[0]

1

In [20]:
len(lista)

3

In [21]:
2 in lista

True

In [41]:
lista = PositiveIntegerList([1, 2, 3])

In [26]:
# list = __builtin__.list

In [29]:
lista[0]

1

In [30]:
len(lista)

3

In [31]:
2 in lista

True

In [36]:
dir(lista)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_data',
 'append']

In [42]:
lista.append(1111)

In [43]:
1111 in lista

True

In [44]:
str(lista)

'<__main__.PositiveIntegerList object at 0x10d21e660>'

## Obekt wywolywany

In [45]:
class Multiplier:
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, x):
        return x * self.factor

In [46]:
times3 = Multiplier(3)

In [47]:
times3(9)

27

## Deskryptor

In [55]:
class PositiveNumber:

    def __set_name__(self, owner, name):
        self.priv_name = f"_{name}"
    
    def __get__(self, instance, owner):
        print(owner)
        return instance.__dict__.get(self.priv_name, 0)

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Value must be positibe")
        instance.__dict__[self.priv_name] = value


class Account:
    balance = PositiveNumber()
    vat = PositiveNumber()

    def __init__(self, balance):
        self.balance = balance
        self.vat = 0


acc = Account(100)
acc.balance

<class '__main__.Account'>


100

In [56]:
vars(acc)

{'_balance': 100, '_vat': 0}

## Cwiczenie:

Zaprojektuj deskryptor Email - ktory waliduje format adresu email i przechowuje go w instancji

In [57]:
"@" in "ala@ma.kota"

True

In [79]:
import re

EMAIL_PATTERN = re.compile(r"^[\w\.]+@\w+.\w+$")

class EmailField:


    def __set_name__(self, owner, name):
        self.priv_name = f"_{name}"
        self.public_name = name

    def __get__(self, instance, owner):
        if instance is None:
            return self
            
        return instance.__dict__.get(self.priv_name)

    def __set__(self, instance, value):
        if not EMAIL_PATTERN.match(value):
            raise ValueError(f"Niepoprawny email w polu {self.name}")
        instance.__dict__[self.priv_name] = value

    def xxx(self):
        print("xxx")

class User:
    email = EmailField()
    def __init__(self, email):
        self.email = email

u = User("adam@example.com")
vars(u)
u.email.xxx()

AttributeError: 'str' object has no attribute 'xxx'

In [71]:
User("xxx")

ValueError: Niepoprawny email w polu email

Stwórz deskryptor `HistoryField`, który zapisuje historię zmian wartości w liście.

In [80]:

class HistoryField:

    def __init__(self, name: str):
        self.name = name
        self.history_name = f"_{name}_history"

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name)

    def __set__(self, instance, value):
        history = instance.__dict__.setdefault(self.history_name, [])
        history.append(value)
        instance.__dict__[self.name] = value

    def history(self, instance):
        return instance.__dict__.get(self.history_name, [])


class Product:
    price = HistoryField("price")

    def __init__(self, price):
        self.price = price




item = Product(10)
item.price = 12
print(item.price) 
print(Product.price.history(item))
[10, 12]


12


AttributeError: 'NoneType' object has no attribute '__dict__'